In [3]:
pip install langchain -q

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install PyPDF2 -q

Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_community.document_loaders import PyPDFLoader

In [11]:
loader = PyPDFLoader(r"C:\Users\alenp\Downloads\infinite_attention.pdf")
pages = loader.load_and_split()
print(type(pages))
print(pages)


<class 'list'>
[Document(page_content='Preprint. Under review.\nLeave No Context Behind:\nEfficient Infinite Context Transformers with Infini-attention\nTsendsuren Munkhdalai, Manaal Faruqui and Siddharth Gopal\nGoogle\ntsendsuren@google.com\nAbstract\nThis work introduces an efficient method to scale Transformer-based Large\nLanguage Models (LLMs) to infinitely long inputs with bounded memory\nand computation. A key component in our proposed approach is a new at-\ntention technique dubbed Infini-attention. The Infini-attention incorporates\na compressive memory into the vanilla attention mechanism and builds\nin both masked local attention and long-term linear attention mechanisms\nin a single Transformer block. We demonstrate the effectiveness of our\napproach on long-context language modeling benchmarks, 1M sequence\nlength passkey context block retrieval and 500K length book summarization\ntasks with 1B and 8B LLMs. Our approach introduces minimal bounded\nmemory parameters and ena

In [2]:
import PyPDF2

In [3]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        # Create a PDF reader object
        pdf_reader = PyPDF2.PdfReader(file)
        
        # Iterate over each page in the PDF
        for page_num in range(len(pdf_reader.pages)):
            # Extract text from the current page
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    
    return text

# Example usage
pdf_path = r"data\infinite_attention.pdf"
pdf_text = extract_text_from_pdf(pdf_path)
print(pdf_text)

Preprint. Under review.
Leave No Context Behind:
Efficient Infinite Context Transformers with Infini-attention
Tsendsuren Munkhdalai, Manaal Faruqui and Siddharth Gopal
Google
tsendsuren@google.com
Abstract
This work introduces an efficient method to scale Transformer-based Large
Language Models (LLMs) to infinitely long inputs with bounded memory
and computation. A key component in our proposed approach is a new at-
tention technique dubbed Infini-attention. The Infini-attention incorporates
a compressive memory into the vanilla attention mechanism and builds
in both masked local attention and long-term linear attention mechanisms
in a single Transformer block. We demonstrate the effectiveness of our
approach on long-context language modeling benchmarks, 1M sequence
length passkey context block retrieval and 500K length book summarization
tasks with 1B and 8B LLMs. Our approach introduces minimal bounded
memory parameters and enables fast streaming inference for LLMs.
1 Introduction
M

In [4]:
print(type(pdf_text))

<class 'str'>


In [7]:
from langchain.text_document import TextDocument

ModuleNotFoundError: No module named 'langchain.text_document'

In [17]:
from langchain_text_splitters import NLTKTextSplitter

text_splitter = NLTKTextSplitter(chunk_size=500, chunk_overlap=100)

chunks = text_splitter.split_documents(pages)

print(len(chunks))

print(type(chunks[0]))

Created a chunk of size 568, which is longer than the specified 500
Created a chunk of size 506, which is longer than the specified 500
Created a chunk of size 633, which is longer than the specified 500


110
<class 'langchain_core.documents.base.Document'>


In [7]:
print(chunks[0])

page_content='Preprint.\n\nUnder review.\n\nLeave No Context Behind:\nEfficient Infinite Context Transformers with Infini-attention\nTsendsuren Munkhdalai, Manaal Faruqui and Siddharth Gopal\nGoogle\ntsendsuren@google.com\nAbstract\nThis work introduces an efficient method to scale Transformer-based Large\nLanguage Models (LLMs) to infinitely long inputs with bounded memory\nand computation.\n\nA key component in our proposed approach is a new at-\ntention technique dubbed Infini-attention.' metadata={'source': 'C:\\Users\\alenp\\Downloads\\infinite_attention.pdf', 'page': 0}


In [12]:
pip install langchain-google-genai -q

Note: you may need to restart the kernel to use updated packages.


In [18]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_model = GoogleGenerativeAIEmbeddings(google_api_key="*****************", 
                                               model="models/embedding-001")

In [40]:
# Store the chunks in vector store
from langchain_community.vectorstores import Chroma

# Embed each chunk and load it into the vector store
db = Chroma.from_documents(chunks, embedding_model, persist_directory="./chroma_db_")

# Persist the database on drive
db.persist()

In [20]:
db_connection = Chroma(persist_directory="./chroma_db_", embedding_function=embedding_model)

In [21]:
# Converting CHROMA db_connection to Retriever Object
retriever = db_connection.as_retriever(search_kwargs={"k": 5})

print(type(retriever))

<class 'langchain_core.vectorstores.VectorStoreRetriever'>


In [30]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [31]:
chat_template = ChatPromptTemplate.from_messages([
    # System Message Prompt Template
    SystemMessage(content="""You are a Helpful AI Bot. 
    You take the context and question from user. Your answer should be based on the specific context."""),
    # Human Message Prompt Template
    HumanMessagePromptTemplate.from_template("""Aswer the question based on the given context.
    Context:
    {context}
    
    Question: 
    {question}
    
    Answer: """)
])


In [35]:
from langchain_google_genai import ChatGoogleGenerativeAI

chat_model = ChatGoogleGenerativeAI(google_api_key="*********************", 
                                   model="gemini-1.5-pro-latest")
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [36]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | chat_template
    | chat_model
    | output_parser
)

In [38]:
response = rag_chain.invoke("What is infinite attention?")

response

'## Infinite Attention Explained:\n\nBased on the context you provided, **Infini-attention** is a novel attention mechanism designed to handle both **long and short-range contextual dependencies** within sequences efficiently. It achieves this by incorporating two key elements into the standard attention mechanism:\n\n* **Compressive Memory:** This allows the model to store and retrieve information from past attention computations, enabling it to access and utilize long-term context effectively. Instead of discarding old key-value (KV) states like standard attention, Infini-attention stores them in the compressive memory. These stored states can then be retrieved using the current query states when processing new sequences, providing access to a much broader context.\n* **Local Causal Attention:** This focuses on capturing dependencies within a local context window, similar to masked attention in standard transformers. This allows the model to efficiently learn relationships between ne

In [39]:
from IPython.display import Markdown as md

md(response)

## Infinite Attention Explained:

Based on the context you provided, **Infini-attention** is a novel attention mechanism designed to handle both **long and short-range contextual dependencies** within sequences efficiently. It achieves this by incorporating two key elements into the standard attention mechanism:

* **Compressive Memory:** This allows the model to store and retrieve information from past attention computations, enabling it to access and utilize long-term context effectively. Instead of discarding old key-value (KV) states like standard attention, Infini-attention stores them in the compressive memory. These stored states can then be retrieved using the current query states when processing new sequences, providing access to a much broader context.
* **Local Causal Attention:** This focuses on capturing dependencies within a local context window, similar to masked attention in standard transformers. This allows the model to efficiently learn relationships between nearby elements in the sequence.

**Essentially, Infini-attention combines the strengths of both local and global context processing, making it well-suited for tasks involving long sequences and requiring an understanding of both recent and historical information.** 


In [41]:
db_connection = Chroma.load("./chroma_db_")

AttributeError: type object 'Chroma' has no attribute 'load'

In [42]:
db_connection.persist("./chroma_db_")

TypeError: Chroma.persist() takes 1 positional argument but 2 were given